In [45]:
from inference_sdk import InferenceHTTPClient
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import pandas as pd
import os 

In [44]:
client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com", 
    api_key="132cxQxyrOVmPD63wJrV" # api keys are individual, change to your own 
)

In [56]:
# input image links 
seal_urls = ["https://media.discordapp.net/attachments/522140586434756611/1341636078993674300/dji_0531.jpg?ex=67b6b777&is=67b565f7&hm=bc213a3e77711770db415a6682bfd3de7a0d2a4e7996b41f56ef864bc6432c1f&=&format=webp&width=1080&height=720"]

In [60]:
clump_imgs_dct = {} # dictionary of clumps. image id will be the key and a list of clumps will be its value. 
num_seals = [] # number of individual seals 

def intersects(seal, clump):
    """Check if two objects (dictionaries) overlap."""
    return not (
        seal['x'] + seal['width'] <= clump['x'] or
        seal['x'] >= clump['x'] + clump['width'] or
        seal['y'] + seal['height'] <= clump['y'] or
        seal['y'] >= clump['y'] + clump['height']
    )

for url in seal_urls:

    response = requests.get(url)
    image = Image.open(BytesIO(response.content))

    result = client.run_workflow(
        workspace_name="elephant-seals-project",
        workflow_id="detect-count-and-visualize-5",
        images={"image": url},
        use_cache=True 
    )

    preds = result[0]['predictions']['predictions']

    seals = [pred for pred in preds if pred['class'] == 'seals']
    clumps = [pred for pred in preds if pred['class'] == 'clump']

    # getting individual seals 
    filtered_seals = [seal for seal in seals if not any(intersects(seal, clump) for clump in clumps)]
    num_seals.append(len(filtered_seals))
    
    # getting clumps 
    clump_imgs_dct[url] = [] 
    for clump in clumps:
        x1_clump = clump['x'] - clump['width'] / 2
        x2_clump = clump['x'] + clump['width'] / 2
        y1_clump = clump['y'] - clump['height'] / 2
        y2_clump = clump['y'] + clump['height'] / 2

        top_left_clump = (x1_clump, y1_clump)
        bottom_right_clump = (x2_clump, y2_clump)

        subimage = image.crop((*top_left_clump, *bottom_right_clump))
        
        clump_imgs_dct[url].append(subimage) 

In [73]:
# Extracting Length, Width and RGB metrics 

keys = []
widths = []
heights = []
avg_r = []
sd_r = []
avg_g = []
sd_g = []
avg_b = []
sd_b = [] 

for key, clump_lst in clump_imgs_dct.items():

    for clump in clump_lst: 

        keys.append(key)
    
        width, height = clump.size

        widths.append(width)
        heights.append(height)

        img_array = np.array(clump)

        avg_r.append(np.mean(img_array[1, :, :]))
        sd_r.append(np.std(img_array[1, :, :]))
        avg_g.append(np.mean(img_array[:, 1, :]))
        sd_g.append(np.std(img_array[:, 1, :]))
        avg_b.append(np.mean(img_array[:, :, 1]))
        sd_b.append(np.std(img_array[:, :, 1]))

heuristics = pd.DataFrame({'key': keys, 
                          'width': widths, 
                          'height': heights,
                          'avg_r': avg_r, 
                          'sd_r': sd_r, 
                          'avg_g': avg_g,
                          'sd_g': sd_g,
                          'avg_b': avg_b,
                          'sd_b': sd_b
                          })

In [74]:
heuristics.head()

,key,width,height,avg_r,sd_r,avg_g,sd_g,avg_b,sd_b
0,https://media.discordapp.net/attachments/52214...,42,44,95.333333,14.293411,107.560606,19.470746,122.245671,41.449524
1,https://media.discordapp.net/attachments/52214...,58,33,105.022989,13.818176,99.404040,13.762722,98.541275,20.956531
2,https://media.discordapp.net/attachments/52214...,42,48,101.865079,14.479517,84.208333,19.991274,120.346230,41.788431
3,https://media.discordapp.net/attachments/52214...,32,57,96.489583,15.163000,110.479532,24.710331,122.048246,36.630323
4,https://media.discordapp.net/attachments/52214...,42,41,124.793651,26.456408,106.902439,15.592063,114.564460,29.859595


In [75]:
# writing heuristics 

heuristics.to_csv('heuristics.csv') # change to filepath 

In [80]:
# saving clumps as images in a new folder  

output_directory = "clumps" # insert folder to save clumps 

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for key, clump_lst in clump_imgs_dct.items(): 
    for idx, img in enumerate(clump_lst):
        img_path = os.path.join(output_directory, f"clump_{idx+1:04d}.jpg") # naming scheme needs to change... incorporate key name somehow 
        img.save(img_path)